# Profiling the save speed

## Operation under investigation

We save a big numpy array in one go into a fresh DB file.

## System information

In [1]:
import qcodes
from qcodes.utils import installation_info as ii

ver = ii.get_qcodes_version()
print(f'QCoDeS version: {ver}')

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\wihpniel\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\wihpniel\.qcodes\logs\200109-12544-qcodes.log
QCoDeS version: 0.9.0rc1+56.g9648416db


In [2]:
ii.get_qcodes_version()

'0.9.0rc1+56.g9648416db'

# Experiment

In [3]:
import tempfile
import time

import numpy as np

from qcodes.dataset.measurements import Measurement
from qcodes.dataset.experiment_container import new_experiment
from qcodes.dataset.sqlite.database import initialise_or_create_database_at


In [4]:
def run_experiment(n):
    """
    Run the experiment with 3*n^2 data points to be saved
    """

    with tempfile.TemporaryFile() as dbfile:
        dbfile.name += '.db'
        print(dbfile.name)
        initialise_or_create_database_at(dbfile.name)
        new_experiment('profiling', 'no_sample')



        meas = (Measurement()
                .register_custom_parameter('X', paramtype='array')
                .register_custom_parameter('Y', paramtype='array')
                .register_custom_parameter('Z', paramtype='array', setpoints=['X', 'Y']))

        t0 = time.perf_counter()

        with meas.run() as datasaver:
            t1 = time.perf_counter()

            xdata, ydata = np.meshgrid(np.random.rand(n), np.random.rand(n))
            zdata = np.random.randn(*xdata.shape)

            t2 = time.perf_counter()
            datasaver.add_result(('X', xdata), ('Y', ydata), ('Z', zdata))
        t3 = time.perf_counter()

    print('Report:')
    print(f'Number of data points saved: {3*n**2}')
    print(f'Init time:            {t1-t0} s')
    print(f'Data generation time: {t2-t0} s')
    print(f'Data saving time:     {t3-t2} s')
    print(f'Total time:           {t3-t0} s')

In [5]:
profile = %prun -r -s time -q datprof = run_experiment(1000)

C:\Users\wihpniel\AppData\Local\Temp\tmpsy6f3mv7.db
Upgrading database; v0 -> v1: : 0it [00:00, ?it/s]
Upgrading database; v1 -> v2: 100%|██████████| 1/1 [00:00<00:00, 249.72it/s]
Upgrading database; v2 -> v3: : 0it [00:00, ?it/s]
Upgrading database; v3 -> v4: : 0it [00:00, ?it/s]
Upgrading database; v4 -> v5: 100%|██████████| 1/1 [00:00<00:00, 143.13it/s]
Upgrading database; v5 -> v6: : 0it [00:00, ?it/s]
Upgrading database; v7 -> v8: 100%|██████████| 1/1 [00:00<00:00, 249.75it/s]
Starting experimental run with id: 1. 
Report:
Number of data points saved: 3000000
Init time:            0.06022890000000025 s
Data generation time: 0.09992409999999996 s
Data saving time:     0.40096440000000033 s
Total time:           0.5008885000000003 s
 

In [6]:
profile

In [7]:
import pstats
from pstats import SortKey
p = profile
p.sort_stats(SortKey.TIME).print_stats(10)


         54474 function calls (51030 primitive calls) in 0.904 seconds

   Ordered by: internal time
   List reduced from 816 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       62    0.506    0.008    0.506    0.008 {method 'commit' of 'sqlite3.Connection' objects}
      179    0.156    0.001    0.195    0.001 {method 'execute' of 'sqlite3.Cursor' objects}
      610    0.045    0.000    0.045    0.000 {method 'acquire' of '_thread.lock' objects}
        1    0.031    0.031    0.031    0.031 {method 'randn' of 'mtrand.RandomState' objects}
       15    0.015    0.001    0.015    0.001 {built-in method numpy.array}
        1    0.013    0.013    0.363    0.363 database.py:200(initialise_database)
        9    0.012    0.001    0.012    0.001 {method 'write' of '_io.BytesIO' objects}
        3    0.012    0.004    0.012    0.004 {method 'tobytes' of 'numpy.ndarray' objects}
        3    0.011    0.004    0.011    0.004 {method 're